# 🧪 MS/MS Spectra Modification Classifier with Transformers

This notebook builds and trains a deep learning model designed to detect and classify *post-translational modifications (PTMs)* in MS/MS spectra from shotgun proteomics. The input is MS/MS spectra in `mgf` format and the classifier is based on a hybrid CNN-Transformer architecture.

---

### 🧠 Objectives

- **Multi-class Classification**: If modified, predict the specific type:
  - Unmodified
  - Oxidation
  - Phosphorylation
  - Ubiquitination
  - Acetylation

---

### 🔧 Environment Setup

The following libraries and paths are configured:

#### 📦 Core Libraries
- `torch`, `torch.nn`, `torch.optim`: PyTorch for neural network construction and training.
- `numpy`, `random`, `os`, `sys`: Utilities for array operations, randomness, and file handling.
- `math`, `datetime`, `logging`: Math functions, timestamping, and logging system.
- `matplotlib.pyplot`: (optional) Visualization.
- `scikit-learn`: Evaluation metrics and dataset splitting.


#### 🛠️ Path Configuration
- Adds the dataset directory on Google Drive to the system path to ensure data files can be accessed during training and evaluation.

---

### 🧬 Pipeline Overview

This project includes the following components:
- **MGF File Parsing**: Custom loader to extract raw spectra from `.mgf` files dataset.
- **Spectral Preprocessing**: Converts spectra into binned, normalized vector representations.
- **Metadata Normalization**: Processes and scales parent ion mass (`pepmass`) for model input.
- **HYbrid CNN-Transformer Model**: Hybrid neural architecture combining CNNs, self-attention, and metadata fusion.
- **Training & Evaluation**: Loop with weighted loss, custom metrics, logging, and model checkpointing.

This setup is tailored for high-performance PTM classification while maintaining compatibility with Google Colab workflows and GPU acceleration tuned using Optuna.



---

## 📁 Directory Setup Instructions

Before running the notebook, ensure your **Google Drive** is properly structured so that the code can:

* Load `.mgf` spectra files.
* Save model weights.
* Persist log files from training.

This is **required** for the notebook to run end-to-end.

---

### 🔗 1. Mount Google Drive

At the beginning of your notebook, run:

```python
from google.colab import drive
drive.mount('/content/drive')
```

You will be prompted to authorize access.

---

### 📂 2. Create This Folder Structure in Your Drive

Organize your files inside `MyDrive` as follows:

```
MyDrive/
├── data/
│   └── balanced_dataset/                ← contains balanced .mgf files for training, they dont neeed to be balanced in the class distribution, but it help in tranning performance
│       ├── split_file_001.mgf
│       ├── split_file_002.mgf
│       └── ...
├── peak_encoder_transformer_pipeline/
│   ├── model_weights/                   ← for saving trained model weights
│   └── logs/                            ← for saving training logs
```

If these folders don't exist, you can create them manually in Google Drive or use Python:

```python
import os

os.makedirs("/content/drive/MyDrive/data/balanced_dataset", exist_ok=True)
os.makedirs("/content/drive/MyDrive/peak_encoder_transformer_pipeline/model_weights", exist_ok=True)
os.makedirs("/content/drive/MyDrive/peak_encoder_transformer_pipeline/logs", exist_ok=True)
```

---

### ⚙️ 3. Update Paths in the Code (if needed)

These variables should point to the correct folders:

```python
input_dir = "your split dataset path"
model_weights_dir = "path for where the weights go"
log_dir = "path for the log system for the per bath logs to be"
```

Make sure the paths you changed to your own are comtable with is expected of each one of them


---

✅ **Once these are set**, you're ready to run the notebook end-to-end, including training, evaluation, and logging.




In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 12.1 MB/s eta 0:00:00


In [ ]:
#Set up the enviorment imports and paths that are necessary for the processing of the cells

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('content/drive/MyDrive/data/balanced_dataset')  # Add the folder containing main.py to sys.path
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import os
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, precision_recall_curve
import matplotlib.pyplot as plt
import logging
from datetime import datetime
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import classification_report
import torch.nn.functional as F
from torch.nn import SiLU
import re
import optuna

Mounted at /content/drive


## 📂 DatasetHandler class for loading MGF files

This section defines the `DatasetHandler` class responsible for managing the loading and iteration over `.mgf` files containing MS/MS spectra.
Loading small `.mgf` at a time in order to make the pipeline scalable without running in to out of memory memory issues.

---

### 📦 `DatasetHandler` Overview

The `DatasetHandler` class provides a memory-efficient way to iterate through `.mgf` files stored in a directory. It supports:

- **Shuffling input files** to randomize data order across training loops.
- **Per-file usage tracking** with `MAX_FILE_PASSES`, ensuring that no file is overused during training.
- **Controlled looping** over the dataset using `num_loops` to allow multiple training epochs without data reloading.

---

### 🧩 Key Components make this under the code explaining how to use it, make it like an example under evrything

#### 🔧 Initialization
```python
handler = DatasetHandler(input_dir="/path/to/mgf", num_loops=1)

In [ ]:
#Setting up the dataset handler class that handles the input
#There is still prints to remove

MAX_FILE_PASSES = 1 # Max times a file can be used before being ignored

class DatasetHandler:
    def __init__(self, input_dir, num_loops=1):
        """
        Initialize the dataset handler.

        Args:
            input_dir (str): Path to the directory containing split MGF files.
            num_loops (int): Number of times the dataset should be iterated.
        """
        self.files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.mgf')]
        self.files = random.sample(self.files, len(self.files))  # Shuffle files
        self.file_usage_counter = {f: 0 for f in self.files}
        self.num_loops = num_loops
        self.loop_count = 0

    def get_next_file(self) -> list:
      """
      Load one MGF file at a time into RAM and return all valid spectra from it.

      Returns:
          list of dict: Each dict contains a valid spectrum and its metadata.
      """
      while self.loop_count < self.num_loops:
          available_files = [f for f in self.files if self.file_usage_counter[f] < MAX_FILE_PASSES]
          if not available_files:
              self.loop_count += 1
              if self.loop_count < self.num_loops:
                  print("Restarting dataset loop...")
                  self.file_usage_counter = {f: 0 for f in self.files}
                  continue
              else:
                  print("All dataset loops completed.")
                  return None

          file = random.choice(available_files)
          print(f"Processing file: {file}")

          spectra = []
          spectrum_data = None

          with open(file, 'r') as f:
              for line in f:
                  line = line.strip()

                  if line == "BEGIN IONS":
                      spectrum_data = {"mz_values": [], "intensity_values": []}

                  elif line.startswith("TITLE=") and spectrum_data is not None:
                      spectrum_data["title"] = line.split("=", 1)[1].strip()

                  elif line.startswith("PEPMASS=") and spectrum_data is not None:
                      try:
                          spectrum_data["pepmass"] = float(line.split("=", 1)[1].split()[0])
                      except ValueError:
                          spectrum_data["pepmass"] = None  # mark as missing
                  elif line.startswith("CHARGE=") and spectrum_data is not None:
                      charge_str = line.split("=", 1)[1].strip()
                      match = re.match(r'^(\d+)', charge_str)  # Match one or more digits at the start
                      if match:
                          spectrum_data["charge"] = int(match.group(1))
                      else:
                          print(f"[SKIPPED CHARGE] Invalid charge format: '{charge_str}'")
                          spectrum_data["charge"] = None

                  elif line == "END IONS" and spectrum_data is not None:
                      title = spectrum_data.get("title", "").strip()
                      mz_vals = spectrum_data.get("mz_values", [])
                      int_vals = spectrum_data.get("intensity_values", [])

                      # Final validation before appending
                      if not title or not title.strip():
                          print(f"[SKIPPED] Missing TITLE in file: {file}")
                      elif not mz_vals or not int_vals:
                          print(f"[SKIPPED] Empty m/z or intensity in file: {file}")
                      elif len(mz_vals) != len(int_vals):
                          print(f"[SKIPPED] Mismatched m/z and intensity count in file: {file}")
                      elif np.sum(int_vals) == 0:
                          print(f"[SKIPPED] All-zero intensities in spectrum '{title}'")
                      else:
                          spectra.append(spectrum_data)

                      spectrum_data = None  # Reset for next spectrum

                  else:
                      if spectrum_data is not None:
                          try:
                              parts = line.split()
                              if len(parts) != 2:
                                  raise ValueError("Expected two float values")
                              mz, intensity = map(float, parts)
                              if math.isnan(mz) or math.isnan(intensity):
                                  raise ValueError("NaN detected")
                              spectrum_data["mz_values"].append(mz)
                              spectrum_data["intensity_values"].append(intensity)
                          except ValueError:
                              #print(f"[SKIPPED LINE] Invalid peak: '{line}' in file: {file}")
                              continue

          self.file_usage_counter[file] += 1

          if spectra:
              return spectra, file

      print("All spectra processed.")
      return None


# ⚙️ Dense Vector Binning for 1D CNN Input  
This section defines the updated preprocessing pipeline for converting annotated MS/MS spectra into dense, fixed-length vectors. These are tailored for use in models such as CNNs or hybrid CNN-Transformer architectures.

## 🔧 Functions:

### `bin_spectra_to_dense_vectors`  
Converts a list of spectra into fixed-length vectors by:  
- **Binning the m/z values** across a specified range (`mz_min` to `mz_max`) into `num_bins`.  
- Each bin holds the **intensity sum of peaks** falling into that m/z range.  
- Applies **sliding window normalization**:  
  The m/z axis is divided into fixed-size windows (e.g., 200 m/z), and intensities within each window are normalized individually to the [0, 1] range. This preserves local signal structure and prevents domination by high-intensity regions.

### `process_spectra_with_handler`  
Processes a batch of spectra by:  
- Logging and skipping spectra with empty or invalid m/z or intensity values.  
- Using the above function to apply binning and **sliding window normalization**.  
- Skipping spectra with no signal after binning (i.e., zero-vector).  

Returns a list of valid, normalized dense vectors for CNN input and logs the total number of skipped spectra.

## 📦 Output Format:  
Each spectrum becomes a 1D `np.array` of shape `(num_bins,)` with `float32` values.  

The final output is either:  
- a stacked `np.ndarray` of shape `(batch_size, num_bins)` when using `bin_spectra_to_dense_vectors` directly on a list, or  
- a list of valid vectors (1 per spectrum) when using `process_spectra_with_handler`.
 used.

In [ ]:
def bin_spectra_to_dense_vectors(spectra_data, num_bins=5000, mz_min=100.0, mz_max=2200.0, window_size=200.0):
    """
    Converts spectra into dense, fixed-length binned vectors suitable for 1D CNN input with sliding window normalization.

    Parameters:
    - spectra_data: List of spectra dicts with 'mz_values' and 'intensity_values'.
    - num_bins: Number of bins to divide the m/z range [mz_min, mz_max] into.
    - mz_min: Minimum m/z value for binning.
    - mz_max: Maximum m/z value for binning.
    - window_size: Size of m/z window for normalization (default is 200.0).

    Returns:
    - np.ndarray of shape (batch_size, num_bins) with per-spectrum normalized intensities.
    """
    bin_edges = np.linspace(mz_min, mz_max, num_bins + 1)
    binned_spectra = []

    for spectrum in spectra_data:
        mz_values = np.array(spectrum['mz_values'])
        intensity_values = np.array(spectrum['intensity_values'])

        if len(mz_values) == 0 or len(intensity_values) == 0:
            binned_spectra.append(np.zeros(num_bins, dtype=np.float32))
            continue

        # Create an array to hold the binned intensities (fixed size)
        binned_intensity = np.zeros(num_bins)

        # Iterate over windows of m/z values
        for window_start in np.arange(mz_min, mz_max, window_size):
            window_end = window_start + window_size
            window_mask = (mz_values >= window_start) & (mz_values < window_end)
            window_mz_values = mz_values[window_mask]
            window_intensity_values = intensity_values[window_mask]

            if len(window_mz_values) > 0:
                # Bin the intensities for this window
                binned_window_intensity, _ = np.histogram(window_mz_values, bins=bin_edges, weights=window_intensity_values)

                # Normalize the binned intensities within this window
                min_val = binned_window_intensity.min()
                max_val = binned_window_intensity.max()
                range_val = max_val - min_val if max_val != min_val else 1e-6
                normalized_binned_window = (binned_window_intensity - min_val) / range_val

                # Add the normalized intensities to the final vector (same size as before)
                binned_intensity += normalized_binned_window

        binned_spectra.append(binned_intensity.astype(np.float32))

    return np.stack(binned_spectra)  # Shape: (batch_size, num_bins)


def process_spectra_with_handler(spectra_batch, num_bins=1000, window_size=200.0):
    """
    Processes spectra batch and returns a list of 1D CNN-ready vectors (one per spectrum),
    with sliding window normalization applied.
    """
    spectrum_vectors = []
    skipped_spectra = 0

    for idx, spectrum in enumerate(spectra_batch):
        title = spectrum.get("title", f"unnamed_{idx}")
        mz_values = np.array(spectrum['mz_values'])
        intensity_values = np.array(spectrum['intensity_values'])

        if mz_values.size == 0 or intensity_values.size == 0:
            print(f"[SKIPPED] Empty m/z or intensity array: '{title}'")
            skipped_spectra += 1
            continue

        # Call the binning function with windowed normalization
        binned_spectrum = bin_spectra_to_dense_vectors([spectrum], num_bins=num_bins, window_size=window_size)

        # Ensure only valid (non-zero) spectra are added
        if np.sum(binned_spectrum) == 0:
            print(f"[SKIPPED] Zero intensity after binning: '{title}'")
            skipped_spectra += 1
            continue

        spectrum_vectors.append(binned_spectrum[0])  # Extract the vector

    print(f"Total skipped spectra: {skipped_spectra}")
    return spectrum_vectors


## 🔬 Normalize Parent Ion Mass (PEPMASS)

This module provides utilities to **extract sequences**, **convert observed m/z to monoisotopic neutral mass** (if needed), and **normalize parent ion values** into the range [0, 1].

---

### 🎯 Objectives (current implementation)

- **Extract** peptide sequence from the beginning of the `TITLE` field.  
- **Convert** PEPMASS from **observed m/z** to **monoisotopic single charged mass** when `assume_observed=True`.  
- **Normalize** the parent ion mass into \[0, 1\] using global bounds from `min_max_dict`.



### 🧩 Key Functions

#### 🔹 `extract_sequence_from_title(title: str) -> str`
Extracts the peptide sequence from the `TITLE`.  
Assumes the sequence is the **first token** (before the first space).

**Example**
```python
TITLE = "GWSMSEQSEESVGGR 2,S,Phospho"
extract_sequence_from_title(TITLE)
# → "GWSMSEQSEESVGGR"
```

🔹 `observed_to_monoisotopic(observed_mz: float, charge: int) -> float`

Converts observed precursor **m/z** into **monoisotopic neutral mass**:

$$
\text{mono\_mass} = z \cdot \text{m/z} - (z - 1)\cdot \text{PROTON\_MASS}
$$

Uses `PROTON_MASS = 1.007276`.

---

#### 🔹 `normalize_parent_ions(data, min_max_dict, assume_observed=True) -> list[float]`

Normalizes parent ion values to the range \$0, 1\$.

* **Inputs per spectrum (dict):**

  * `"pepmass"`: precursor value
  * `"charge"`: integer charge state

* **Behavior:**

  1. If `assume_observed=True`:

     * Converts `"pepmass"` (observed m/z) → monoisotopic neutral mass.
  2. If `assume_observed=False`:

     * Uses `"pepmass"` directly (assumed monoisotopic).
  3. Normalizes with:

     $$
     \text{norm} = \frac{parent\_ion - min}{max - min}
     $$
  4. Clamps results into \$0, 1\$.
  5. Missing metadata → returns `0.0`.

**Example**

```python
min_max = {"min": 400.0, "max": 6000.0}
normalized = normalize_parent_ions(spectra, min_max, assume_observed=True)
```

---

### ✅ Output

Returns:

```python
[List of float values between 0 and 1]
```

---

### ⚠️ Notes

* Requires `"min"` and `"max"` keys in `min_max_dict`.
* Missing or invalid metadata defaults to **0.0**.
* No theoretical mass calculation or spectrum validation is performed here.

In [ ]:

PROTON_MASS = 1.0072764665789
H2O_MASS = 18.01056

def extract_sequence_from_title(title: str) -> str:
    """
    Extracts the peptide sequence from the TITLE string.
    Assumes the sequence is the first word, before the first space.
    """
    if not isinstance(title, str) or not title.strip():
        return ""
    return title.strip().split(" ")[0]  # safe even with extra spaces



def observed_to_monoisotopic(observed_mz, charge):
    return charge * observed_mz - (charge - 1) * PROTON_MASS



def normalize_parent_ions(data, min_max_dict, assume_observed=True):
    """
    Normalize parent ions to the range [0, 1].

    If assume_observed=True, converts PEPMASS (observed m/z) to monoisotopic mass before computing normalization.
    """
    normalized = []

    for spectrum in data:
        pepmass = spectrum.get("pepmass", None)
        charge = spectrum.get("charge", None)

        if pepmass is None or charge is None:
            normalized.append(0.0)
            continue

        if assume_observed:
            mono_mass = observed_to_monoisotopic(pepmass, charge)
            parent_ion = mono_mass
        else:
            parent_ion = pepmass  # Already monoisotopic

        # Normalize to [0, 1]
        pepmass_min = min_max_dict["min"]
        pepmass_max = min_max_dict["max"]
        norm = (parent_ion - pepmass_min) / (pepmass_max - pepmass_min)
        normalized.append(max(0, min(1, norm)))

    return normalized



### 🧬 Combine Spectra with Parent Ion Mass, change the model to always recieve monoistopic single charged mass inetas of obserd mass like we currently do.


This function constructs the final **input representation** for the neural network by pairing each processed spectrum with its corresponding normalized parent ion mass.

---

### ⚙️ `combine_features(...)`

#### **Purpose**
Aggregates spectral and precursor metadata into a unified format, ready to be passed into the model during training or evaluation.

---

### 🔄 Process Flow

1. **Spectral Preprocessing**
   - Calls `process_spectra_with_handler(...)` to:
     - Apply binning and normalization.
     - Generate a dense, fixed-length vector for each spectrum.
   - Result: `spectra_vectors` — a list of shape `[batch_size, num_bins]`.

2. **Parent Ion Normalization**
   - Invokes `normalize_parent_ions(...)` to:
     - Convert precursor monoisotopic mass to observed mass.
     - Normalize to a range of [0, 1] using dataset-specific bounds.
   - Result: `parent_ions` — a list of length `[batch_size]`.

3. **Validation**
   - Verifies alignment between spectrum vectors and parent ion list.
   - Logs an error and aborts if lengths mismatch.

4. **Zipping**
   - Combines each spectrum vector and its corresponding normalized parent ion into a tuple:
     ```python
     (spectrum_vector, normalized_parent_ion)
     ```

---

### 📤 Output Format

```python
[
  (spectrum_vec₁, pepmass₁),
  (spectrum_vec₂, pepmass₂),
  ...
]



In [ ]:
def combine_features(data, pepmass_min_max, num_bins, window_normaliation_size, assume_observed):
    """
    Converts spectra + metadata into model input tuples:
        (binned spectrum, normalized parent ion mass)
    """

    spectra_vectors = process_spectra_with_handler(data, num_bins, window_normaliation_size)
    if not spectra_vectors:
        return None

    parent_ions = parent_ions = normalize_parent_ions(
    data, pepmass_min_max, assume_observed=assume_observed)


    if len(spectra_vectors) != len(parent_ions):
        print("❌ Mismatch between spectra and parent ions.")
        return None

    return list(zip(spectra_vectors, parent_ions))



### 🏷️ Label Spectra Based on Modifications

This function performs **automatic labeling** of MS/MS spectra for supervised learning, based on the content of the `TITLE` field in each spectrum's metadata.

---

### 🧠 Purpose

Assigns integer labels to each spectrum in a batch according to the presence of post-translational modification (PTM) keywords in the title:

- `0` → **Unmodified**
- `1` → **Oxidation** (if the string `"oxidation"` appears in the title)
- `2` → **Phosphorylation** (if the string `"phospho"` appears in the title)
- `3` → **Ubiquitination** (if the string `"k_gg"` appears in the title)
- `4` → **Acetylation** (if the string `"k_ac"` appears in the title)

The result is a list of labels aligned with the order of input spectra — suitable for classification tasks using `softmax`
---

### ⚙️ Logic

For each spectrum in the input list:
1. Checks that the entry is a dictionary.
2. Extracts the `title` and converts it to lowercase.
3. Searches for PTM-related keywords.
4. Defaults to `0` if no match or invalid format.

---

### 📤 Output Format

Returns:
```python
[0, 2, 1, 4, 3, ...]


In [ ]:
#This cell reads the labels of the data and prepares it for the model


#Faltam bue labels agr

def spectrum_label(spectra_data) -> list:
    """
    Assigns labels to spectra based on known modifications in TITLE.

    Parameters:
    - spectra_data (list of dict): List of spectrum dictionaries (from DatasetHandler).

    Returns:
    - List of labels for each spectrum.
    """
    if not isinstance(spectra_data, list):
        print("ERROR: Expected a list of spectra, got", type(spectra_data))
        return None

    labels = []

    for spectrum in spectra_data:
        if not isinstance(spectrum, dict):
            print(f"WARNING: Expected spectrum to be a dict, got {type(spectrum)}")
            labels.append(0)
            continue

        # Get spectrum title and verify it's a non-empty string
        spectrum_id = spectrum.get("title", "")
        if not isinstance(spectrum_id, str) or not spectrum_id.strip():
            print(f"WARNING: Missing or invalid title for spectrum, assigning label 0")
            labels.append(0)
            continue

        spectrum_id = spectrum_id.lower().strip()  # Normalize for label detection

        # Assign labels based on keywords in TITLE
        if "oxidation" in spectrum_id:
            labels.append(1)
        elif "phospho" in spectrum_id:
            labels.append(2)
        elif "k_gg" in spectrum_id:
            labels.append(3)
        elif "k_ac" in spectrum_id:
            labels.append(4)
        else:
            labels.append(0)

    print(f"Labels: {Counter(labels)}")
    return labels


---

# 🧠 Hybrid CNN-Transformer Classification Model

This module defines the **final architecture** used for **multi-class PTM classification** from MS/MS spectra.
The model integrates **local pattern extraction (CNN)**, **global context modeling (Transformer)**, and **metadata (parent ion mass)** into a single classification head.

---

## 🔹 `PositionalEncoding`

Implements **sinusoidal positional encodings** (Vaswani et al., 2017), injecting sequence order information into embeddings.

* **Signature:**

  ```python
  PositionalEncoding(d_model: int = 64, seq_len: int = 175, dropout: float = 0.1)
  ```
* **Behavior:** Precomputes a tensor of `sin`/`cos` terms and adds it to input, followed by dropout.
* **Input:** `[B, L, d_model]` with `L ≤ seq_len`.
* **Output:** Same shape as input.

**Example**

```python
pe = PositionalEncoding(d_model=64, seq_len=1000, dropout=0.1)
x = torch.randn(32, 10, 64)   # [batch, seq_len, d_model]
x = pe(x)                     # same shape
```

---

## 🔹 `EncoderTransformerClassifier`

A **hybrid classifier** with four main blocks:

1. **1D CNN Encoder** – Extracts local spectral patterns

   ```
   Conv1d(1→32, k=5, pad=2) → BN → ReLU
   MaxPool1d(k=2)            # halves length
   Conv1d(32→64, k=3, pad=1) → BN → ReLU
   Flatten
   ```

   * **Output:** `[B, 64 * (input_size // 2)]`

2. **Linear Encoder** – Projects CNN features into Transformer latent space

   ```
   Linear(64*(S/2) → 512) → BN → ReLU
   Linear(512 → latent_size) → BN → ReLU → Dropout
   ```

   * **Output:** `[B, latent_size]`

3. **Positional Encoding + Transformer** – Global context

   * Expand to sequence: `[B, 1, latent_size]`
   * Add sinusoidal encoding
   * Pass through `nn.TransformerEncoder` (`num_layers`, `num_heads`, `dim_feedforward=4*latent_size`)
   * Mean over sequence dim → `[B, latent_size]`

4. **Parent Ion Processor** – Encodes normalized parent mass

   ```
   Linear(1 → 64) → ReLU
   Linear(64 → latent_size) → ReLU
   ```

   * **Output:** `[B, latent_size]`

5. **Fusion & Classification**

   ```
   concat([spectrum, parent]) → [B, 2*latent_size]
   Dropout
   Linear(2*latent_size → num_classes)
   ```

   * **Output:** logits `[B, num_classes]`

---

### ✅ Forward Pass

**Inputs**

* `spectra`: `[B, S]` (dense binned spectrum, length = `input_size`)
* `parent_ion`: `[B]` (normalized precursor mass)

**Output**

* `logits`: `[B, num_classes]`

---

### 🔧 Implementation Notes

* `latent_size % num_heads == 0` is enforced.
* `input_size` must be **even** (due to `MaxPool1d`).
* The Transformer currently sees only one token per spectrum (global embedding). To use **true attention over multiple tokens**, pass a sequence (e.g., CNN feature map before flattening).

---

### 🧪 Example

```python
model = EncoderTransformerClassifier(
    input_size=175, latent_size=64, num_classes=5,
    num_heads=4, num_layers=2, dropout_prob=0.1
)

spectra = torch.randn(32, 175)   # [batch, S]
parent  = torch.rand(32)         # [batch], normalized
logits  = model((spectra, parent))  # [32, 5]
```

**Loss**

* Multi-class PTM classification:

  ```python
  loss_fn = nn.CrossEntropyLoss()
  loss = loss_fn(logits, labels)   # labels ∈ [0..num_classes-1]


In [ ]:


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int = 64, seq_len: int = 175, dropout: float = 0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)

        position = torch.arange(seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(seq_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))  # shape: [1, seq_len, d_model]

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len]
        return self.dropout(x)


class EncoderTransformerClassifier(nn.Module):
    def __init__(self, input_size, latent_size, num_classes, num_heads, num_layers, dropout_prob, max_len=1000):
        super(EncoderTransformerClassifier, self).__init__()
        self.input_size = input_size
        self.latent_size = latent_size
        self.num_classes = num_classes

        # Validate divisibility
        if latent_size % num_heads != 0:
            raise ValueError(f"latent_size ({latent_size}) must be divisible by num_heads ({num_heads}).")

        # 1. CNN Encoder (New Layer)
        self.cnn_encoder = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),  # Downsample
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Flatten()
        )

        # 2. Linear Encoder (Refactored)
        self.encoder = nn.Sequential(
            nn.Linear(64 * (input_size // 2), 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, latent_size),
            nn.BatchNorm1d(latent_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob)
        )

        #3. Positional Encoding
        self.positional_encoding = PositionalEncoding(d_model=latent_size, seq_len=max_len, dropout=dropout_prob)

        # 4. Transformer Encoder
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=latent_size,
                nhead=num_heads,
                dim_feedforward=latent_size * 4,
                dropout=dropout_prob,
                activation='relu',
                batch_first=True,
                norm_first=False
            ),
            num_layers=num_layers
        )

        # Parent Ion Layer
        self.parent_ion_layer = nn.Sequential(
            nn.Linear(1, 64),
            nn.ReLU(),
            nn.Linear(64, self.latent_size),
            nn.ReLU()
        )

        # Dropout before classification
        self.dropout = nn.Dropout(dropout_prob)


        self.classifier_head = nn.Linear(latent_size * 2, num_classes)


    def forward(self, inputs):
        spectra, parent_ion = inputs
        parent_ion = parent_ion.unsqueeze(1)

        # CNN Encoder
        spectra = spectra.unsqueeze(1)  # Ensure input is [B, 1, S]
        cnn_output = self.cnn_encoder(spectra)

        # Linear Encoder
        x = self.encoder(cnn_output)

        # Positional Encoding and Transformer
        x = x.unsqueeze(1)  # Adding sequence dimension
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)

        # Parent Ion Encoding
        parent = self.parent_ion_layer(parent_ion).squeeze(1)

        # Concatenate
        combined = torch.cat([x, parent], dim=1)
        combined = self.dropout(combined)

        logits = self.classifier_head(combined)  # shape: [batch, num_classes]
        return logits





### 🧪 Training, Evaluation & Logging Utilities

This section documents the **current** training, evaluation, and logging helpers for the Transformer-based classifier on MS/MS spectra. It replaces older notes that referred to multi-label BCE and PR/ROC AUC.

---

#### 🗂️ Logging Setup
Configures a clean logging pipeline for Colab:

- **Clears** any pre-existing root handlers to prevent duplicate logs.
- **Persists** run logs to Google Drive at (change to the path you want the directory of your log file)
```

/content/drive/MyDrive/4\_mod\_model/logs/4\_mod\_transformer.log

````
- Uses a named logger: `spectra_logger` with level **INFO**.

---

#### 🧪 `train_classifier_with_weights(...)`

Trains the classifier with **multi-class CrossEntropyLoss** and optional **L1 regularization**.

**Key features**
- **Optimizer:** `AdamW` with `weight_decay=0.01`.
- **Loss:** `CrossEntropyLoss` on **integer class labels** (not one-hot).
- **Class imbalance:** Per-class counts are computed; a weight vector is prepared **but not applied** by default.  
> To enable class weighting, pass it to CE:  
> `loss_fn = nn.CrossEntropyLoss(weight=class_weights)`.
- **Regularization:** Optional **L1** penalty via `l1_lambda`.
- **Metrics:** Per-epoch **loss** and **accuracy**; prints debug samples every 10 epochs.
- **Checkpointing:** Saves `state_dict` to `save_path` when training completes and if the costum scoring fuction imporves, the user can also set a minium score so the model doesnt uncessary save in the beginning of tranning, making it so it only save high quality results.

**Inputs**
- `model`: the `EncoderTransformerClassifier`.
- `data_tensors`: `(spectra_tensors[B, S], parent_ions[B])`.
- `labels`: integer tensor `[B]` with values in `[0, num_classes-1]`.
- Typical kwargs: `epochs=100`, `learning_rate=1e-3`, `l1_lambda=1e-3`, `device='cuda'`.

**Output**
- Saves weights to disk and prints/logs training progress.

---

#### 📊 `evaluate_model(...)`

Evaluates the model on a validation batch using **multi-class** metrics.

**What it does**
- Runs inference with `CrossEntropyLoss` against integer labels.
- Computes:
- **Macro / Weighted:** Precision, Recall, F1
- **Accuracy**
- **Per-class report:** via `classification_report`
- **Class distribution** (counts)
- **Composite score** (returned) prioritizes balanced performance under imbalance:

\[
\text{Score} \;=\; 0.45\cdot \text{Macro-F1} \;+\; 0.30\cdot \text{Macro-Recall} \;+\; 0.15\cdot \text{Weighted-F1} \;+\; 0.10\cdot \text{Accuracy}
\]

- Logs a formatted summary (including per-class table) via `spectra_logger`.

**Inputs**
- `model`, `data_tensors`, `labels` (same shapes as training).
- `batch` (optional): tag to identify the batch in logs.

**Output**
- **Returns:** the composite score (float).

---

### ✅ Notes & Assumptions

- **Task type:** This pipeline is **multi-class**, not multi-label. Use `CrossEntropyLoss` targets of shape `[B]`.
- **Class names:** The list
```python
["Unmodified", "Oxidation", "Phospho", "Ubiquitination", "Acetylation"]
````

should match your `num_classes`. Update it if you use a different class set/order.

* **Imbalance handling:** Class weights are computed but **not applied** unless passed to `CrossEntropyLoss`.
* **AUC metrics:** PR-AUC / ROC-AUC are **not** computed in the current implementation. Add them explicitly if needed.
* **Parent ion input:** Expected normalized to `[0, 1]` (see the normalization module).

---

### 🔧 Minimal Usage Example

```python
# Train
score_path = "model_final_weights.pth"
train_classifier_with_weights(
    model, (spectra_tensors, parent_ions), labels,
    epochs=50, learning_rate=1e-3, l1_lambda=1e-3,
    save_path=score_path, device='cuda'
)

# Evaluate
val_score = evaluate_model(model, (val_spectra, val_parent_ions), val_labels, batch="val-01")
print("Validation composite score:", val_score)


In [ ]:
#Testing, evaluating and logging cell

#Remove existing handlers to prevent Colab caching issues
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

log_dir = "/content/drive/MyDrive/4_mod_model/logs"
os.makedirs(log_dir, exist_ok=True)

log_path = os.path.join(log_dir, "4_mod_transformer.log")

# Reapply config AFTER handlers are cleared
logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

logger = logging.getLogger("spectra_logger")



def train_classifier_with_weights(model, data_tensors, labels, epochs=100, learning_rate=0.001,l1_lambda=0.001, save_path="model_final_weights.pth", device='cuda'):
    """
    Trains the encoder-classifier model with a weighted loss function for handling class imbalance.
    Assumes all inputs are already PyTorch tensors on the correct device.
    """
    spectra_tensors, parent_ions = data_tensors
    parent_ions = parent_ions.unsqueeze(1)  # Ensure shape (batch_size, 1)
    labels_tensors = labels  # Already a tensor

    # Compute class weights (just once on CPU for Counter)
    class_counts = Counter(labels_tensors.cpu().numpy())
    num_classes = torch.max(labels_tensors).item() + 1
    class_weights = torch.ones(num_classes, dtype=torch.float32).to(device)

    for cls in range(num_classes):
        if cls in class_counts:
            class_weights[cls] = len(labels_tensors) / (num_classes * class_counts[cls])

    print(f"Class Weights: {class_weights.cpu().numpy()}")

    # Optimizer & Loss
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01) #SGD worth a try?
    loss_fn = nn.CrossEntropyLoss()
     # or add pos_weight=torch.tensor([...]) if it performs badly

    epoch_losses = []
    epoch_accuracies = []

    # Train loop
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        outputs = model((spectra_tensors ,parent_ions))
        loss = loss_fn(outputs, labels_tensors)

        # L1 Regularization
        l1_loss = 0.0
        for param in model.parameters():
            l1_loss += torch.sum(torch.abs(param))
        loss += l1_lambda * l1_loss

        loss.backward()
        optimizer.step()


        predictions = torch.argmax(outputs, dim=1)
        accuracy = (predictions == labels_tensors).float().mean().item()


        #Store metrics
        epoch_losses.append(loss.item())
        epoch_accuracies.append(accuracy)

        #Log epoch details
        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {loss.item():.4f}, Accuracy: {accuracy * 100:.2f}%")

        # Debugging Information
        if (epoch + 1) % 10 == 0 or epoch == epochs - 1:
            print(f"Sample Predictions: {predictions[:5].cpu().numpy()}")
            print(f"Actual Labels: {labels[:5]}")
            print(f"Sample Logits: {outputs[:5].detach().cpu().numpy()}")

    #Save model weights after training
    torch.save(model.state_dict(), save_path)
    print(f"Final model weights saved to {save_path}")



def evaluate_model(model, data_tensors, labels, batch=None):

    model.eval()
    spectra_tensors, parent_ions = data_tensors
    parent_ions = parent_ions.unsqueeze(1)


    targets = labels.long()


    with torch.no_grad():
        outputs = model((spectra_tensors, parent_ions))
        loss_fn = nn.CrossEntropyLoss()  # if you're using them
        loss = loss_fn(outputs, targets)

        predictions = torch.argmax(outputs, dim=1)

        predictions_np = predictions.cpu().numpy()
        targets_np = targets.cpu().numpy()

        print("predictions device:", predictions.device)
        print("targets device:", targets.device)

        # Macro & Weighted scores
        macro_precision = precision_score(targets_np, predictions_np, average='macro', zero_division=0)
        macro_recall = recall_score(targets_np, predictions_np, average='macro', zero_division=0)
        macro_f1 = f1_score(targets_np, predictions_np, average='macro', zero_division=0)
        weighted_precision = precision_score(targets_np, predictions_np, average='weighted', zero_division=0)
        weighted_recall = recall_score(targets_np, predictions_np, average='weighted', zero_division=0)
        weighted_f1 = f1_score(targets_np, predictions_np, average='weighted', zero_division=0)

        accuracy = (predictions == targets).float().mean().item()


        # Class distribution
        class_distribution = Counter(targets_np)


        # Per-class report
        class_names = ["Unmodified", "Oxidation", "Phospho", "Ubiquitination", "Acetylation"]
        report = classification_report(targets_np, predictions_np, target_names=class_names, zero_division=0, digits=4)


        score = (
            0.45 * macro_f1 +
            0.30 * macro_recall +
            0.15 * weighted_f1 +
            0.10 * accuracy
        )


        log_message = (
            f"Batch {batch if batch is not None else '-'}: Validation Loss: {loss.item():.4f},\n"
            f"Macro Precision: {macro_precision:.4f}, Macro Recall: {macro_recall:.4f}, Macro F1-score: {macro_f1:.4f},\n"
            f"Weighted Precision: {weighted_precision:.4f}, Weighted Recall: {weighted_recall:.4f}, Weighted F1-score: {weighted_f1:.4f},\n"
            f"Accuracy: {accuracy * 100:.2f}%, "
            f"Class Distribution: {class_distribution}\n"
            f"Per-class metrics:\n{report}"
        )

        print(log_message)
        logger.info(log_message)

    return  score

here’s a paste-ready markdown tailored to **this** 5-class Optuna script.

# Optuna Hyperparameter Tuning (5-Class, Streaming Mini-Batches)

We tune the Transformer classifier with **Optuna** using **low-memory, file-streamed mini-batches** from `.mgf` files. The objective is to maximize a validation score computed consistently with the project’s evaluation function.

---

## Setup

* **Classes:** `num_classes = 5`
* **Data:** `input_dir = "/content/drive/MyDrive/data/4_mod_balanced_dataset"`
* **Artifacts:** `model_weights_dir = "/content/drive/MyDrive/4_mod_model/model_weights"`
* **Precursor normalization:** `pepmass_range = [500.0, 6000.0]`, window `200.0`
* **Device:** GPU if available (`torch.cuda.is_available()`)

### Controller flags

* `run_optuna = True` (run tuning)
* `n_trials = 100` (can be increased)
* `assume_observed = True`, `load_latest_model_at_start = True` (kept for parity with training loop; not used inside objective)

---

## Search Space


Model/training hyperparameters tuned:

* `latent_size ∈ {64, 128, 256}`
* `num_heads ∈ {2, 4, 8}` with constraint **`latent_size % num_heads == 0`** (else prune)
* `num_layers ∈ [2, 6]` (int)
* `dropout_prob ∈ [0.10, 0.35]`
* `l1_lambda ∈ [1e−7, 5e−6]` (log-scaled)
* `learning_rate ∈ [1e−4, 3e−4]` (log-scaled)

---

## Objective & Data Flow (Low-Memory)

Each trial evaluates the candidate hyperparameters over **two independent streamed mini-batches** (two `.mgf` files), reducing variance without loading the full dataset.

1. **Deterministic seeding per trial**
   `seed = trial.number + 1337`; we seed Python, NumPy, and PyTorch (CPU/GPU).

2. **Mini-batch streaming**
   A `DatasetHandler` yields one file at a time (`NUM_BATCHES = 2`).
   For each file:

   * Build features with `combine_features(...)` →
     **(a)** binned spectrum vector of size `num_bins`, **(b)** normalized parent ion.

3. **Per-batch stratified split (80/20)**
   `train_test_split(..., stratify=labels, random_state=seed + batch_index)` ensures class balance within the batch.

4. **Inner training loop with pruning checkpoints**

   * Initialize a fresh `EncoderTransformerClassifier` per trial.
   * Train for **60 epochs** in **3 chunks of 20**:

     * after each 20 epochs, compute `val_score = evaluate_model(...)`
     * report to Optuna via `trial.report(val_score, step=ep)`
     * allow early stopping via `trial.should_prune()`

5. **Trial objective**
   The trial’s score is the **mean** of the validation scores across the available mini-batches.

---

## Pruning

We use **MedianPruner** (`n_warmup_steps = 5`) to terminate underperforming trials early based on intermediate results, saving compute time.

---

## Running the Study & Diagnostics

* Create and run the study:

  ```python
  study = optuna.create_study(direction="maximize",
                              pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
  study.optimize(objective, n_trials=100)  # script currently uses 100 here
  ```
* After optimization, we report the best params and show standard Optuna plots:

  * Parameter Importances
  * Optimization History
  * Slice Plot
  * Parallel Coordinate Plot

These visualizations highlight sensitive hyperparameters and interactions.

---

## Reproducibility

* Fixed random seeds per trial and batch.
* Stratified splits preserve class balance within each streamed mini-batch.
* The attention head divisibility constraint (`latent_size % num_heads == 0`) prevents invalid architectures (invalid trials are pruned immediately).

---

## Why This Design

* **Memory-efficient:** only one `.mgf` file is in memory at a time.
* **Variance-aware:** averaging over two distinct files stabilizes validation estimates.
* **Compute-savvy:** chunked training with intermediate evaluation enables effective **pruning**.
* **Consistent:** uses the same feature pipeline and evaluation routine as the main training workflow.



In [ ]:
# === CONFIGURATION FLAGS ===
run_optuna = True  # Set to False if you want to run the full training loop instead
n_trials = 100  # Number of Optuna trials

# === FIXED PARAMS ===
num_classes = 5
pepmass_range = {'min': 500.0, 'max': 6000.0}
window_normaliation_size = 200.00
epoch = 100
num_loops = 1
min_score_threshold = 0.90
input_dir = "/content/drive/MyDrive/data/4_mod_balanced_dataset"
model_weights_dir = "/content/drive/MyDrive/4_mod_model/model_weights"
assume_observed = True
load_latest_model_at_start = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))

def objective(trial):
    # --- Fixed or 2-stage: pick one ---
    num_bins = 4500
    # --- Model hyperparams ---
    latent_size  = trial.suggest_categorical("latent_size", [64, 128, 256])
    num_heads    = trial.suggest_categorical("num_heads", [2, 4, 8])
    if latent_size % num_heads != 0:
        raise optuna.exceptions.TrialPruned()

    num_layers   = trial.suggest_int("num_layers", 2, 6)
    dropout_prob = trial.suggest_float("dropout_prob", 0.1, 0.35)   # narrower to avoid underfit with L1
    l1_lambda    = trial.suggest_float("l1_lambda", 1e-7, 5e-6, log=True)
    lr           = trial.suggest_float("learning_rate", 1e-4, 3e-4, log=True)

    # --- Determinism per trial ---
    seed = trial.number + 1337
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

    # --- Evaluate on multiple mini-batches to reduce variance ---
    NUM_BATCHES = 2  # or 3 if affordable
    scores = []

    handler = DatasetHandler(input_dir, num_loops=1)

    for b in range(NUM_BATCHES):
        batch = handler.get_next_file()
        if batch is None:
            break
        spectra_batch, file_path = batch

        # (Optional) group-aware split: skip here if you can’t extract groups quickly
        feature_batch = combine_features(spectra_batch, pepmass_range, num_bins,
                                         window_normaliation_size, assume_observed)
        if not feature_batch:
            raise optuna.exceptions.TrialPruned()

        spectra, parent_ions = zip(*feature_batch)
        X_spec = torch.tensor(np.array(spectra), dtype=torch.float32, device=device)
        X_pi   = torch.tensor(np.array(parent_ions), dtype=torch.float32, device=device)
        y      = torch.tensor(spectrum_label(spectra_batch), dtype=torch.long, device=device)

        idx_tr, idx_va = train_test_split(np.arange(len(y)), test_size=0.2,
                                          stratify=y.detach().cpu().numpy(),
                                          random_state=seed + b)

        train_data = (X_spec[idx_tr], X_pi[idx_tr])
        val_data   = (X_spec[idx_va], X_pi[idx_va])
        y_tr, y_va = y[idx_tr], y[idx_va]

        model = EncoderTransformerClassifier(
            input_size=num_bins, latent_size=latent_size, num_heads=num_heads,
            num_layers=num_layers, dropout_prob=dropout_prob, num_classes=num_classes
        ).to(device)

        # Shorter inner training + report for pruning
        EPOCHS = 60
        for ep in range(0, EPOCHS, 20):
            train_classifier_with_weights(
                model, train_data, y_tr,
                epochs=20, learning_rate=lr, l1_lambda=l1_lambda,
                save_path="/dev/null", device=device
            )
            val_score = evaluate_model(model, val_data, y_va)
            trial.report(val_score, step=ep+20)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        scores.append(val_score)

    if not scores:
        raise optuna.exceptions.TrialPruned()

    return float(np.mean(scores))



study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials= 100)

print("Best trial:")
print(study.best_trial.params)

optuna.visualization.plot_param_importances(study).show()
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_slice(study).show()
optuna.visualization.plot_parallel_coordinate(study).show()

[I 2025-09-16 02:03:04,239] A new study created in memory with name: no-name-7b11e50a-d00c-4c58-8b60-feb9bc1486fd


CUDA available: True
Device: NVIDIA L4
Processing file: /content/drive/MyDrive/data/4_mod_balanced_dataset/split_file_037.mgf
Total skipped spectra: 0
Labels: Counter({0: 359, 1: 350, 3: 251, 2: 176, 4: 66})
Class Weights: [0.66968644 0.68642855 1.3631206  0.961      3.626415  ]
Epoch [1/20] - Loss: 2.1109, Accuracy: 22.37%
Epoch [2/20] - Loss: 1.7459, Accuracy: 46.62%
Epoch [3/20] - Loss: 1.6220, Accuracy: 55.46%
Epoch [4/20] - Loss: 1.5544, Accuracy: 58.79%
Epoch [5/20] - Loss: 1.4790, Accuracy: 64.00%
Epoch [6/20] - Loss: 1.4041, Accuracy: 70.66%
Epoch [7/20] - Loss: 1.3374, Accuracy: 73.78%
Epoch [8/20] - Loss: 1.2524, Accuracy: 78.67%
Epoch [9/20] - Loss: 1.1753, Accuracy: 82.52%
Epoch [10/20] - Loss: 1.0771, Accuracy: 86.47%
Sample Predictions: [1 3 0 1 3]
Actual Labels: tensor([1, 0, 0, 1, 3], device='cuda:0')
Sample Logits: [[-0.20299911  1.0040014   0.14009853  0.05741541 -0.73667336]
 [ 0.09131964 -0.38636446  0.23621047  1.3219966  -1.7356995 ]
 [ 1.8972821   1.2484958   0.1

/usr/local/lib/python3.12/dist-packages/optuna/trial/_trial.py:501: UserWarning: The reported value is ignored because this `step` 20 is already reported.
  warnings.warn(


Epoch [1/20] - Loss: 0.5740, Accuracy: 98.23%
Epoch [2/20] - Loss: 0.5450, Accuracy: 98.75%
Epoch [3/20] - Loss: 0.5170, Accuracy: 99.17%
Epoch [4/20] - Loss: 0.4918, Accuracy: 99.38%
Epoch [5/20] - Loss: 0.4730, Accuracy: 99.17%
Epoch [6/20] - Loss: 0.4556, Accuracy: 99.58%
Epoch [7/20] - Loss: 0.4407, Accuracy: 99.79%
Epoch [8/20] - Loss: 0.4289, Accuracy: 100.00%
Epoch [9/20] - Loss: 0.4209, Accuracy: 99.90%
Epoch [10/20] - Loss: 0.4097, Accuracy: 99.69%
Sample Predictions: [4 3 0 3 1]
Actual Labels: tensor([4, 3, 0, 3, 1], device='cuda:0')
Sample Logits: [[ 0.6491272  -1.0670115  -0.9472309  -0.6647108   4.0056014 ]
 [-1.298186   -2.4734921  -1.9847916   4.116307   -0.19793013]
 [ 4.081837   -1.1361313  -0.48404986 -0.8910955  -0.10031009]
 [-1.1658239  -1.1422902  -2.6387804   4.7766995  -0.8030215 ]
 [-2.760413    4.479639   -0.33274096 -1.5148419   0.25053257]]
Epoch [11/20] - Loss: 0.4013, Accuracy: 99.90%
Epoch [12/20] - Loss: 0.3969, Accuracy: 99.90%
Epoch [13/20] - Loss: 0.3

/usr/local/lib/python3.12/dist-packages/optuna/trial/_trial.py:501: UserWarning: The reported value is ignored because this `step` 40 is already reported.
  warnings.warn(


...

Final model weights saved to /dev/null
predictions device: cuda:0
targets device: cuda:0
Batch -: Validation Loss: 1.8520,
Macro Precision: 0.4603, Macro Recall: 0.2098, Macro F1-score: 0.1117,
Weighted Precision: 0.4427, Weighted Recall: 0.3071, Weighted F1-score: 0.1545,
Accuracy: 30.71%, Class Distribution: Counter({np.int64(0): 72, np.int64(1): 71, np.int64(3): 51, np.int64(2): 34, np.int64(4): 13})
Per-class metrics:
                precision    recall  f1-score   support

    Unmodified     0.3013    1.0000    0.4630        72
     Oxidation     0.0000    0.0000    0.0000        71
       Phospho     1.0000    0.0294    0.0571        34
Ubiquitination     1.0000    0.0196    0.0385        51
   Acetylation     0.0000    0.0000    0.0000        13

      accuracy                         0.3071       241
     macro avg     0.4603    0.2098    0.1117       241
  weighted avg     0.4427    0.3071    0.1545       241

Processing file: /content/drive/MyDrive/data/4_mod_balanced_dataset

[I 2025-09-16 02:33:40,435] Trial 98 pruned. 


Final model weights saved to /dev/null
predictions device: cuda:0
targets device: cuda:0
Batch -: Validation Loss: 3.4227,
Macro Precision: 0.1603, Macro Recall: 0.2027, Macro F1-score: 0.0979,
Weighted Precision: 0.2415, Weighted Recall: 0.3029, Weighted F1-score: 0.1464,
Accuracy: 30.29%, Class Distribution: Counter({np.int64(1): 73, np.int64(0): 72, np.int64(3): 50, np.int64(2): 33, np.int64(4): 13})
Per-class metrics:
                precision    recall  f1-score   support

    Unmodified     0.3013    1.0000    0.4630        72
     Oxidation     0.5000    0.0137    0.0267        73
       Phospho     0.0000    0.0000    0.0000        33
Ubiquitination     0.0000    0.0000    0.0000        50
   Acetylation     0.0000    0.0000    0.0000        13

      accuracy                         0.3029       241
     macro avg     0.1603    0.2027    0.0979       241
  weighted avg     0.2415    0.3029    0.1464       241

Processing file: /content/drive/MyDrive/data/4_mod_balanced_dataset

[I 2025-09-16 02:33:51,995] Trial 99 pruned. 


Final model weights saved to /dev/null
predictions device: cuda:0
targets device: cuda:0
Batch -: Validation Loss: 2.5242,
Macro Precision: 0.1930, Macro Recall: 0.2083, Macro F1-score: 0.1121,
Weighted Precision: 0.1887, Weighted Recall: 0.3029, Weighted F1-score: 0.1522,
Accuracy: 30.29%, Class Distribution: Counter({np.int64(0): 72, np.int64(1): 70, np.int64(3): 50, np.int64(2): 36, np.int64(4): 13})
Per-class metrics:
                precision    recall  f1-score   support

    Unmodified     0.2983    0.9861    0.4581        72
     Oxidation     0.0000    0.0000    0.0000        70
       Phospho     0.6667    0.0556    0.1026        36
Ubiquitination     0.0000    0.0000    0.0000        50
   Acetylation     0.0000    0.0000    0.0000        13

      accuracy                         0.3029       241
     macro avg     0.1930    0.2083    0.1121       241
  weighted avg     0.1887    0.3029    0.1522       241

Best trial:
{'latent_size': 64, 'num_heads': 4, 'num_layers': 6, 'd